In [ ]:
# Place interactive figures inside the notebook
%matplotlib widget
#matplotlib inline

# Import modules
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact

# Create vector images using TikZ?
#
# To use TikZ, make sure that ImageMagick and pdf2svg packages are installed
# on your machine, then install "ipython-tikzmagic" jupyter extension with
#
# pip3 install --user git+git://github.com/mkrphys/ipython-tikzmagic.git
use_tikz = True

# Load TikZ extension, or create dummy magic to skip cell content
if use_tikz:
    %load_ext tikzmagic
else:
    from IPython.core.magic import register_cell_magic
    @register_cell_magic
    def tikz(line, cell=None):
        return

# Maxwell's equations on a periodic 1D grid of $[0,L]$

Consider the homogeneous 1D Maxwell equations on our periodic domain

\begin{equation}
\left\{
\begin{split}
\frac{\partial E}{\partial t} + \frac{\partial B}{\partial x} &= 0, \\
\frac{\partial B}{\partial t} + \frac{\partial E}{\partial x} &= 0,
\end{split}
\right.
\end{equation}

with initial conditions $E(t=0,x) = E_0(x)$ and $B(t=0,x) = B_0(x)$.

## Exact solution

By summing or subtracting Amperè's and Faraday's laws, one obtains two decoupled scalar advection equations

\begin{equation}
\left(\frac{\partial}{\partial t} + \frac{\partial}{\partial x}\right) v_1 = 0,
\qquad
\left(\frac{\partial}{\partial t} - \frac{\partial}{\partial x}\right) v_2 = 0,
\end{equation}

where we have introduced the fields

\begin{equation}
\begin{matrix}
v_1 := E + B \\
v_2 := E - B
\end{matrix}
\qquad
\text{so that}
\qquad
\begin{matrix}
E = \dfrac{v_1 + v_2}{2} \\
B = \dfrac{v_1 - v_2}{2}
\end{matrix}
\end{equation}

The exact solution to these equations are traveling waves with propagation velocity $\lambda_{1} = 1$ and $\lambda_{2} = -1$, respectively:

\begin{equation}
v_1(t,x) = v_1(0,x-t),
\qquad
v_2(t,x) = v_2(0,x+t).
\end{equation}

Clearly, $v_{1,2} = E \pm B$ are the __eigenvectors__ of the 1D Maxwell equations, and $\lambda_{1,2} = \pm 1$ are the corresponding __eigenvalues__.

It is now possible to write the exact solution of the 1D Maxwell equations by converting the initial conditions $E(0,x)$ and $B(0,x)$ into the eigenvectors $v_{1,2}(0,x)$, then solving for $v_{1,2}(t,x)$, and finally converting back to $E(t,x)$ and $B(t,x)$. These three steps can be combined to give

\begin{align}
E(t,x) &= \frac{1}{2}\big[ E_0(x-t) + E_0(x+t) \big]
        + \frac{1}{2}\big[ B_0(x-t) - B_0(x+t) \big], \\
B(t,x) &= \frac{1}{2}\big[ B_0(x-t) + B_0(x+t) \big]
        + \frac{1}{2}\big[ E_0(x-t) - E_0(x+t) \big].
\end{align}

In [ ]:
def exact_solution(E0, B0):
    E = lambda t, x : 0.5 * (E0(x - t) + E0(x + t) + B0(x - t) - B0(x + t))
    B = lambda t, x : 0.5 * (B0(x - t) + B0(x + t) + E0(x - t) - E0(x + t))
    return E, B

In [ ]:
# Domain
x = np.linspace(0, 1, 101)

# Number of periods in domain
a = 3

# Callable functions E(t,x) and B(t,x)
E, B = exact_solution(
    E0 = lambda x: np.cos((a*2*np.pi) * x),
    B0 = lambda x: 0.0*np.cos((a*2*np.pi) * x)
)

def plot_exact_solution(t):
    fig = plt.figure(10)
    ax = fig.axes[0] if len(fig.axes) == 1 else fig.add_subplot(1, 1, 1)
    while len(ax.lines) > 0:
        ax.lines.pop()
    ax.plot(x, E(t, x), 'b', label='E')
    ax.plot(x, B(t, x), 'r--', label='B')
    ax.legend(loc='upper right')
    ax.set_ylim([-1.05, 1.05])
    ax.set_xlabel('x')
    ax.grid(True)
    fig.canvas.draw()

plt.close(10)
time_widget = widgets.FloatSlider(min=0, max=1.0, step=0.01, value=0)
interact(plot_exact_solution, t=time_widget)

## Staggered grid

Consider the periodic 1D interval $[0, L]$. The primal grid consists of the nodes of a uniform grid of $N$ cells $x_j = j L~/N$ for $0\le j < N$, and the dual grid consists of the midpoints of the cells $x_{j+1/2} = \left(j+\frac{1}{2}\right) L~/N$ for $0\le j < N$.

In [ ]:
%%tikz -s 600,200 -f svg
\scriptsize
\filldraw[fill=white] (-1,-1) rectangle (6,1);
\draw[step=1.0,black,thin] (0,0) grid (5.,0.);
\foreach \x in {0,...,4}
    \draw (\x cm,-3pt) -- (\x cm,3pt);
\foreach \x in {0.5,...,4.5}      
    \draw (\x,0) node{$\times$};
\foreach \x in {0,...,1}
    \draw (\x,-10pt) node{$x_\x$};
\draw (5.01,3pt) arc (160:200:10pt);
\draw (4,-10pt) node{$x_{N-1}$};  
\draw (0,10pt) node{$0$};  
\draw (5,10pt) node{$1$};        
\draw (0.5,-10pt) node{$x_{\frac{1}{2}}$};
\draw (1.5,-10pt) node{$x_{\frac{3}{2}}$};
\draw (4.7,-10pt) node{$x_{N-\frac{1}{2}}$};

## Yee's scheme

Given a time-step size $\Delta t$, a staggered grid is also created in time:

1. The primal grid has values $t_n = n\Delta t$ for $n\in\mathbb{N}$;
2. The dual grid has values $t_{n+1/2} = \left(n +\frac{1}{2}\right)\Delta t$ for $n\in\mathbb{N}$.

Moreover:

- The field $E$ is described by its point values on the ___primal___ space-time grid $(t_n, x_j)$;
- The field $B$ is described by its point values on the ___dual___ space-time grid $(t_{n+1/2}, x_{j+1/2})$;
- The time and spatial derivatives of $E$ and $B$ are approximated by 2nd-ordered centered finite difference formulas.

We obtain therefore the following update formulas, to be applied consecutively:

\begin{align}
%
\color{orange}{B^{n+1/2}_{j+1/2}} &= B^{n-1/2}_{j+1/2} +
    \frac{\Delta t}{\Delta x}\left(E^{n}_{j} - E^{n}_{j+1}\right),
    \qquad {0\le j < N},\\
%
\color{cyan}{E^{n+1}_{j}} &= E^n_{j} + \frac{\Delta t}{\Delta x}
    \left(\color{orange}{B^{n+1/2}_{j-1/2}} - \color{orange}{B^{n+1/2}_{j+1/2}} \right),
    \qquad {0\le j < N}.
%
\end{align}


### Startup and shutdown

Notice that at time $n=0$ one needs to compute the values $\left\{B^{-1/2}_{j+1/2}\right\}_j$ from the initial condition $\left\{B^{0}_{j+1/2}\right\}_j$ in order to start the scheme:

\begin{equation}
  B^{-1/2}_{j+1/2} = B^0_{j+1/2} - \frac{\Delta t}{2\Delta x}
  \left(E^{0}_{j} - E^{0}_{j+1}\right),
  \qquad {0\le j < N}.
\end{equation}

Similarly, at the last time step $M$ one needs to compute the final solution $\left\{B^{M}_{j+1/2}\right\}_j$ from the previous values $\left\{B^{M-1/2}_{j+1/2}\right\}_j$:

\begin{equation}
  B^{M}_{j+1/2} = B^{M-1/2}_{j+1/2} + \frac{\Delta t}{2\Delta x}
  \left(E^{M}_{j} - E^{M}_{j+1}\right),
  \qquad {0\le j < N}.
\end{equation}

In general, the operation of pushing the discrete $B$ field forward (and then backward) in time is required whenever the solution needs to be visualized.


### Vector formulation

If we define the (circulant) differentiation matrix $D \in \mathbb{R}^{N \times N}$ as

\begin{equation}
  D = \frac{\Delta t}{\Delta x}~
  \begin{pmatrix}
     1 &  0 & \dots  & \dots  & -1 \\
    -1 &  1 &        &        &  0 \\
     0 & -1 &  1     &        &    \\
       &    & \ddots & \ddots &    \\
     0 &    & \ddots &   -1   &  1 \\
  \end{pmatrix}
\end{equation}

we can write the Yee scheme as

\begin{align}
%
\color{orange}{B^{n+1/2}} &= B^{n-1/2} + D^T~E^{n},\\
%
\color{cyan}{E^{n+1}} &= E^n - D~\color{orange}{B^{n+1/2}}.
%
\end{align}

### Energy conservation

The electromagnetic energy in the continuous system at time $t$ is

\begin{equation}
  W(t) = \frac{1}{2} \int_0^L \left[ E^2(t) + B^2(t) \right] dx,
\end{equation}

which is constant in time thanks to the periodic boundary conditions. The Yee scheme cannot conserve the continuous energy $W$, but it exactly preserves a discrete energy $\mathcal{W} \approx W$ which will be defined as follows. Let's define the $L^2$ inner product

\begin{equation}
\langle u, v \rangle_h = \Delta x \sum_{j=0}^{N-1} u_j v_j,
\end{equation}

and notice that the dual operator of $D$ with respect to such inner product is $D^{T}$, that is

\begin{equation}
\langle D u, v \rangle_h = \langle u, D^{T} v \rangle_h.
\end{equation}

Let's consider the Yee update equations in vector form. If we take the inner product of the first equation with $\left(B^{n+1/2}+B^{n-1/2}\right)$, and the inner product of the second equation with $\left(E^{n+1}+E^{n}\right)$, after rearranging the terms we obtain

\begin{align}
\langle B^{n+1/2}, B^{n+1/2} \rangle &= 
\langle B^{n-1/2}, B^{n-1/2} \rangle +
\langle B^{n+1/2}, D^{T}E^{n} \rangle +
\langle B^{n-1/2}, D^{T}E^{n} \rangle
\\
\langle E^{n+1}, E^{n+1} \rangle +
\langle D~B^{n+1/2}, E^{n+1}\rangle &= 
\langle E^{n}, E^{n} \rangle -
\langle D~B^{n+1/2}, E^{n} \rangle
\end{align}

Summing these two equations side-by-side, and noticing that $\langle D~B^{n+1/2}, E^{n}\rangle = \langle B^{n+1/2}, D^{T}E^{n} \rangle$, we obtain

\begin{equation}
\big\| E^{n+1} \big\| +
\big\| B^{n+1/2} \big\| +
\left\langle D~B^{n+1/2}, E^{n+1}\right\rangle
\,=\,
\big\| E^{n} \big\| +
\big\| B^{n-1/2} \big\| +
\left\langle D~B^{n-1/2}, E^{n} \right\rangle
\end{equation}

which can be written as an _exact conservation_ $\mathcal{W}^{n+1} = \mathcal{W}^{n}$ for the discrete energy

\begin{equation}
\boxed{
\mathcal{W}^{n} :=
\frac{1}{2} \big\| E^{n} \big\|^2 +
\frac{1}{2} \big\| B^{n-1/2} \big\|^2 +
\frac{1}{2} \left\langle D~B^{n-1/2}, E^{n} \right\rangle
}
\end{equation}

Hence, the discrete energy $\mathcal{W}^{n}$ as defined above is exactly conserved by the Yee scheme. An equivalent formula that uses the magnetic field at two successive time steps is

\begin{equation}
\mathcal{W}^{n} :=
\frac{1}{2} \big\langle E^{n}, E^{n} \big\rangle +
\frac{1}{2} \big\langle B^{n-1/2}, B^{n+1/2} \big\rangle
\end{equation}

Using the formulas for the startup and shutdown of the Yee scheme, this can be rewritten as

\begin{equation}
\mathcal{W}^{n} :=
\frac{1}{2} \big\langle E^{n}, E^{n} \big\rangle +
\frac{1}{2} \big\langle B^{n} - \frac{1}{2}D^T E^{n},
B^{n} + \frac{1}{2} D^T E^{n}\big\rangle.
\end{equation}

Any of the three equivalent formulas can be used in the code as an energy diagnostic.

In [ ]:
%run ex1_yee.py -h

In [ ]:
%run ex1_yee.py 100 -t 100 -p 1

### Convergence Analysis

In [ ]:
# Coarsest grid: number of cells and time-steps
N0 = 20
s0 = 20

refinement = [(2**i) for i in range(4)]

N_list   = []
err_list = []

for r in refinement:
    N = N0 * r
    s = s0 * r
    
    %run ex1_yee.py $N -p 0 -t $s
    
    N_list.append(yee_namespace['N'])
    err_list.append(yee_namespace['error_E'])

In [ ]:
print_error_analysis(N_list, err_list)

### Stability

If we define the (circulant) differentiation matrix $D \in \mathbb{R}^{N \times N}$ as

\begin{equation}
  D = \frac{\Delta t}{\Delta x}~
  \begin{pmatrix}
     1 &  0 & \dots  & \dots  & -1 \\
    -1 &  1 &        &        &  0 \\
     0 & -1 &  1     &        &    \\
       &    & \ddots & \ddots &    \\
     0 &    & \ddots &   -1   &  1 \\
  \end{pmatrix}
\end{equation}

it is possible to write the one-step update formula of the Yee scheme as

\begin{equation}
  \begin{pmatrix}
  I & 0 \\
  D & I \\
  \end{pmatrix}
  \begin{pmatrix}
    B^{n+1/2} \\
    E^{n+1}   \\
  \end{pmatrix} =
  \begin{pmatrix}
  I & D^{T} \\
  0 & I   \\
  \end{pmatrix}
  \begin{pmatrix}
    B^{n-1/2} \\
    E^{n}     \\
  \end{pmatrix}.
\end{equation}

We notice that the matrix on the left-hand side is easily invertible,

\begin{equation}
  \begin{pmatrix}
  I & 0 \\
  D & I \\
  \end{pmatrix}^{-1} =
  \begin{pmatrix}
   I & 0 \\
  -D & I \\
  \end{pmatrix},
\end{equation}

and therefore

\begin{equation}
  \begin{pmatrix}
    B^{n+1/2} \\
    E^{n+1}   \\
  \end{pmatrix} =
  \begin{pmatrix}
   I &      D^T \\
  -D & I -D D^T   \\
  \end{pmatrix}
  \begin{pmatrix}
    B^{n-1/2} \\
    E^{n}     \\
  \end{pmatrix}.
\end{equation}

Given the initial conditions, the solution after $M$ time steps is obtained by applying the formula above recursively for $M$ times. This yields

\begin{equation}
  \begin{pmatrix}
    B^{M-1/2} \\
    E^{M}   \\
  \end{pmatrix} =
  \begin{pmatrix}
   I &      D^T \\
  -D & I -D D^T   \\
  \end{pmatrix}^{M}
  \begin{pmatrix}
    B^{-1/2} \\
    E^{0}     \\
  \end{pmatrix}.
\end{equation}

If we simplify the notation as

\begin{equation}
  U^{n+1} = A~U^n
\end{equation}

and we write the eigendecomposition $A = Q\Lambda Q^{-1}$ then we have

\begin{equation}
  U^{M} = A^M U^0 = (Q~\Lambda Q^{-1})^M ~ U^0 = Q~\Lambda^M Q^{-1} ~ U^0.
\end{equation}

So clearly the solution $U^M$ will not "explode" as $M\to\infty$ if and only if all the eigenvalues of $A$ satisfy

\begin{equation}
  | \lambda_i | \le 1.
\end{equation}

One can easily see that the __stability__ of the scheme depends the ratio $C_p = \Delta t/\Delta x$, which is called the Courant parameter.

In [ ]:
def yee_eigenvalues(N, Cp):

    from scipy import linalg as la

    I = np.identity(N)
    c = np.zeros(N); c[0] = Cp; c[1] = -Cp
    D = la.circulant(c)
    A = np.block([[ I, D.T               ],
                  [-D, I - np.dot(D, D.T)]])
    
    return np.linalg.eigvals(A)

In [ ]:
abs( yee_eigenvalues(10, 0.7) )

In [ ]:
abs( yee_eigenvalues(10, 1.1) )

In [ ]:
fig = plt.figure(20)
ax  = fig.subplots(1, 1)
ax.grid(True)
unit_circle = plt.Circle((0, 0), 1.0, color='r')
ax.add_artist(unit_circle)

def plot_yee_eigenvalues(N, Cp):
    eigs = yee_eigenvalues(N, Cp)
    if len(ax.lines) > 0:
        ax.lines.pop()
    ax.plot(eigs.real, eigs.imag, '*b')
    ax.set_xlim([-2, 2])
    ax.set_ylim([-2, 2])
    ax.set_aspect('equal')
    ax.set_xlabel(r'$Re(\lambda)$')
    ax.set_ylabel(r'$Im(\lambda)$')
    fig.canvas.draw()

ncells_widget  = widgets.IntSlider(min=10, max=100, step=10, value=10)
courant_widget = widgets.FloatSlider(min=0.1, max=1.2, step=0.1, value=0.5)
interact(plot_yee_eigenvalues, N=ncells_widget, Cp=courant_widget)

## Method of lines (MOL)

In [ ]:
%run ex1_mol.py -h

In [ ]:
%run ex1_mol.py 20 -t 1 -p 0 -f 2 -o 2

In [ ]:
# Coarsest grid: number of cells and time-steps
N0 = 20
s0 = 20

refinement = [(2**i) for i in range(4)]

N_list   = []
err_list = []

for r in refinement:
    N = N0 * r
    s = s0 * r
    
    %run ex1_mol.py $N -p 0 -t $s -f 6 -o 6
    
    N_list.append(mol_namespace['N'])
    err_list.append(mol_namespace['error_E'])

In [ ]:
print_error_analysis(N_list, err_list)